In [11]:
import requests
import time
import re
import openpyxl
import pandas as pd
import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup
from datetime import date

In [12]:
GLOBAL = 'https://www.gipernn.ru'
LINK = 'https://www.gipernn.ru/torgovye-centry?per-page=50'

Добыча колличества ссылок

In [13]:
html = requests.get(LINK)
soup = BeautifulSoup(html.text, 'html.parser')
count = soup.find('div', class_='count')

count = int(count.text.split(':')[1].strip())
page_count = int(np.ceil(count / 50))

print(count)

143


Составляем массив ссылок

In [14]:
links = []
for page in tqdm(range(page_count + 1), ascii=True):
    html = requests.get(f"{LINK}&page={page}")
    soup = BeautifulSoup(html.text, 'html.parser')
    trs = soup.find('tbody').find_all('tr')
    for tr in trs:
        link = tr.find('a')
        if link:
            link = link.get('href')
            link = GLOBAL + link
            links.append(link)
    links = list(set(links))
print(len(links))   


100%|##########| 4/4 [00:00<00:00,  4.30it/s]

143


Составление своей базы данных

In [15]:
df = pd.DataFrame()
for link in tqdm(links, ascii=True):
    html = requests.get(link)
    soup = BeautifulSoup(html.text, 'html.parser')

    tmp = {} # словарь для временных данных
    tmp['Ссылка'] = link

    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    tmp['Заголовок'] = header

    price = soup.find('div', class_ = 'price')
    if price:
        price = price.text.replace('\xa0', ' ').strip().split('  ')
    tmp['Цена'] = price

    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    tmp['Описание'] = description
    
    try:
        w0 = soup.find('table', id = 'w0')
        if w0:
            w0 = w0.find_all('tr')
            for tr in w0:
                key = tr.find('th').text.strip()
                val = tr.find('td').text.strip()
                tmp[key] = val
    except:
        print('w0 not found')
 
    df = pd.concat([df, pd.DataFrame(tmp, index=[0])], ignore_index=True)

df['Источник'] = 'gipernn'
df['Сегмент'] = 'Комерческая недвижимость'
df['Подсегмент'] = 'Аренда торговых центров'
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

100%|##########| 143/143 [00:41<00:00,  3.46it/s]


Посмотрим на результаты вкрадце

In [16]:
df.head()

,Ссылка,Заголовок,Цена,Описание,Источник,Сегмент,Подсегмент,Город,Дата парсинга
0,https://www.gipernn.ru/torgovye-centry/nagorny...,"Торговый центр Нагорный на пл. Советская, 3Сов...",None,None,gipernn,Комерческая недвижимость,Аренда торговых центров,Нижний Новгород,2022-12-22
1,https://www.gipernn.ru/torgovye-centry/assol-u...,"Торговый центр Ассоль на ул. Ленина, 278Богоро...",None,None,gipernn,Комерческая недвижимость,Аренда торговых центров,Нижний Новгород,2022-12-22
2,https://www.gipernn.ru/torgovye-centry/skala-g...,"Торговый центр Скала на ул. Германа Лопатина, ...",None,None,gipernn,Комерческая недвижимость,Аренда торговых центров,Нижний Новгород,2022-12-22
3,https://www.gipernn.ru/torgovye-centry/sokol-u...,"Торговый центр Сокол на ул. Чаадаева, 5дМосков...",None,None,gipernn,Комерческая недвижимость,Аренда торговых центров,Нижний Новгород,2022-12-22
4,https://www.gipernn.ru/torgovye-centry/burnako...,Торговый центр Бурнаковский 3 на ул. Бурнаковс...,None,None,gipernn,Комерческая недвижимость,Аренда торговых центров,Нижний Новгород,2022-12-22


Экспортируем таблицу

In [17]:
df.to_excel('tradecenter-rent.xlsx', index=False)